In [1]:
import numpy as np
import pandas_ta as ta
import pandas as pd
from datetime import datetime, timedelta, timezone
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

In [2]:
def data_backtest(data):
    new_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    new_data.index = pd.to_datetime(data['Date'])
    new_data.index.name = None
    return new_data

def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

def process_data(df):
    data = df.copy()
    data = data[~data.index.duplicated(keep='first')] # Handling duplicate
    
    data['Date'] = [str(i)[:10] for i in data.index]
    data['time'] = [str(i)[11:] for i in data.index]
    data_model = data.pivot(index = 'Date', columns = 'time', values = ['Open','High','Low','Close','Volume']).ffill(axis = 1).stack().reset_index() # Handling missing values

    return data_model

In [3]:
df_min = pd.read_csv('vn30f1m_3min.csv')
df_hour = pd.read_csv('vn30f1m_1hour.csv')
df_day = pd.read_csv('vn30f1m_1day.csv')

data_min = data_backtest(df_min)
data_hour = data_backtest(df_hour)
data_day = data_backtest(df_day)

train_min, valid_min = split_data(data_min)
train_hour, valid_hour = split_data(data_hour)
train_day, valid_day = split_data(data_day)

# 1-day strategies

## Breakout

### Breakout with Bollinger Bands

In [265]:
import talib
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Bollinger Breakout Strategy
class BollingerBreakout(Strategy):
    bb_period = 20
    risk_reward_ratio = 2
    min_diff = 0.001  # Giữ lại biến này nếu nó giúp cải thiện chiến lược

    def init(self):
        self.upper_band, self.middle_band, self.lower_band = self.I(
            talib.BBANDS, self.data.Close, timeperiod=self.bb_period, nbdevup=1.5, nbdevdn=1.5, matype=0)

    def next(self):
        price_diff = self.data.Close[-1] - self.lower_band[-1]
        
        if price_diff > self.min_diff:
            sl = self.data.Close[-1] - price_diff
            tp = self.data.Close[-1] + price_diff * self.risk_reward_ratio
            
            # Không giới hạn vị thế nếu điều đó cải thiện lợi nhuận
            # if not self.position:

            if crossover(self.data.Close, self.upper_band):
                self.buy(size=1/6, sl=sl, tp=tp)
            elif crossover(self.lower_band, self.data.Close):
                self.sell(size=1/6, sl=tp, tp=sl)


# Hàm backtest với hàm optimize
def run_backtest_optimize(strategy_class, train_data, val_data, bb_period_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        bb_period=bb_period_range,  # Phạm vi tối ưu bb_period
        maximize='Return [%]',  # Tối ưu lợi nhuận Return [%]
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation với thông số tối ưu
    print("Backtesting on validation set...")
    optimized_bb_period = stats_train._strategy.bb_period
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run(bb_period=optimized_bb_period)
    
    return stats_train, stats_val

# Example: Chạy trên dữ liệu train_day và valid_day
bb_period_range = range(10, 101, 5)  # Điều chỉnh khoảng giá trị bb_period
stats_train, stats_val = run_backtest_optimize(BollingerBreakout, train_day, valid_day, bb_period_range)

Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/10 [00:00<?, ?it/s]

Backtesting on validation set...


In [ ]:
stats_train

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   75.672981
Equity Final [$]                  7335.553353
Equity Peak [$]                   10465.17442
Return [%]                         144.518445
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                   25.471283
Volatility (Ann.) [%]               61.718677
Sharpe Ratio                           0.4127
Sortino Ratio                        0.793527
Calmar Ratio                         0.631687
Max. Drawdown [%]                  -40.322626
Avg. Drawdown [%]                   -9.035423
Max. Drawdown Duration      602 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                   67
Win Rate [%]                        53.731343
Best Trade [%]                      33.606481
Worst Trade [%]                    -10.594087
Avg. Trade [%]                    

In [267]:
stats_val 

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   84.431138
Equity Final [$]                  3349.360121
Equity Peak [$]                   4186.480961
Return [%]                          11.645337
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                    5.720687
Volatility (Ann.) [%]               37.419241
Sharpe Ratio                         0.152881
Sortino Ratio                        0.221652
Calmar Ratio                          0.15597
Max. Drawdown [%]                  -36.678165
Avg. Drawdown [%]                  -13.960664
Max. Drawdown Duration      359 days 00:00:00
Avg. Drawdown Duration       87 days 00:00:00
# Trades                                   24
Win Rate [%]                             62.5
Best Trade [%]                      16.268519
Worst Trade [%]                     -10.39247
Avg. Trade [%]                    

### Breakout with Volume

In [269]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy

class VolumeBreakout(Strategy):
    risk_reward_ratio = 2  # Tỷ lệ R:R = 1:2
    volume_period = 20  # Số phiên trung bình khối lượng
    stop_loss_pct = 0.02  # Stop Loss là 2% dưới/ trên giá vào lệnh
    take_profit_pct = 0.04  # Take Profit là 4% trên/ dưới giá vào lệnh

    def init(self):
        # Chuyển đổi khối lượng thành kiểu float64 để phù hợp với talib
        volume_float = self.data.Volume.astype(np.float64)
        # Tính khối lượng trung bình
        self.volume_avg = self.I(talib.SMA, volume_float, timeperiod=self.volume_period)

    def next(self):
        # Kiểm tra khối lượng hiện tại có vượt khối lượng trung bình không
        if self.data.Volume[-1] > self.volume_avg[-1]:
            if not self.position:  # Kiểm tra xem có vị thế nào đang mở không
                # Lệnh mua
                if self.data.Close[-1] > self.data.Open[-1]:
                    sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # SL là 2% dưới giá hiện tại
                    tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # TP là 4% trên giá hiện tại
                    self.buy(size=1/6, sl=sl, tp=tp)

                # Lệnh bán
                elif self.data.Close[-1] < self.data.Open[-1]:
                    sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # SL là 2% trên giá hiện tại
                    tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # TP là 4% dưới giá hiện tại
                    self.sell(size=1/6, sl=sl, tp=tp)


# Hàm để thực hiện tối ưu hóa và backtest
def run_backtest_optimize(strategy_class, train_data, val_data, volume_period_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        volume_period=volume_period_range,  # Phạm vi tối ưu volume_period
        maximize='Return [%]',  # Tối ưu Sharpe Ratio
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage
volume_period_range = range(10, 101, 5)  # Điều chỉnh khoảng giá trị volume_period
stats_train, stats_val = run_backtest_optimize(VolumeBreakout, train_day, valid_day, volume_period_range)


Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/10 [00:00<?, ?it/s]

Backtesting on validation set...


In [270]:
stats_train

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   55.433699
Equity Final [$]                      936.364
Equity Peak [$]                        3000.0
Return [%]                         -68.787867
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                  -25.583163
Volatility (Ann.) [%]               13.475918
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -68.787867
Avg. Drawdown [%]                  -68.787867
Max. Drawdown Duration     1420 days 00:00:00
Avg. Drawdown Duration     1420 days 00:00:00
# Trades                                  106
Win Rate [%]                        33.018868
Best Trade [%]                       4.313239
Worst Trade [%]                     -7.641856
Avg. Trade [%]                    

In [271]:
stats_val

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   78.043912
Equity Final [$]                     2004.418
Equity Peak [$]                        3000.0
Return [%]                         -33.186067
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                  -18.425185
Volatility (Ann.) [%]               15.973301
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -34.8026
Avg. Drawdown [%]                    -34.8026
Max. Drawdown Duration      700 days 00:00:00
Avg. Drawdown Duration      700 days 00:00:00
# Trades                                   64
Win Rate [%]                          42.1875
Best Trade [%]                       4.073912
Worst Trade [%]                     -5.220411
Avg. Trade [%]                    

### Breakout with RSI

In [149]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Chiến thuật Break-out với RSI có Stop Loss, Take Profit và kích thước 1/6
class RSI_Breakout(Strategy):
    rsi_period = 14
    rsi_lower = 35  # Điều chỉnh ngưỡng quá bán để tăng số lượng giao dịch
    rsi_upper = 65  # Điều chỉnh ngưỡng quá mua để tăng số lượng giao dịch
    stop_loss_pct = 0.02  # Stop Loss 2%
    take_profit_pct = 0.04  # Take Profit 4%

    def init(self):
        # Tính chỉ báo RSI
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Mua khi RSI dưới ngưỡng quá bán
        if not self.position and self.rsi[-1] < self.rsi_lower:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)
            self.buy(size=1/6, sl=sl, tp=tp)  # Mở lệnh mua với Stop Loss và Take Profit

        # Bán khi RSI vượt ngưỡng quá mua
        elif not self.position and self.rsi[-1] > self.rsi_upper:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)
            self.sell(size=1/6, sl=sl, tp=tp)  # Mở lệnh bán với Stop Loss và Take Profit


# Hàm để thực hiện backtest và tối ưu hóa
def run_backtest_optimize(strategy_class, train_data, val_data, rsi_period_range, rsi_lower_range, rsi_upper_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        rsi_period=rsi_period_range,  # Phạm vi tối ưu rsi_period
        rsi_lower=rsi_lower_range,  # Phạm vi tối ưu rsi_lower
        rsi_upper=rsi_upper_range,  # Phạm vi tối ưu rsi_upper
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        constraint=lambda params: params.rsi_lower < params.rsi_upper,  # Điều kiện hợp lệ cho RSI
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage
rsi_period_range = range(10, 31, 5)  # Điều chỉnh khoảng giá trị rsi_period
rsi_lower_range = range(20, 46, 5)  # Điều chỉnh khoảng giá trị rsi_lower
rsi_upper_range = range(60, 86, 5)  # Điều chỉnh khoảng giá trị rsi_upper

# Thực hiện backtest và tối ưu hóa
stats_train, stats_val = run_backtest_optimize(RSI_Breakout, train_day, valid_day, rsi_period_range, rsi_lower_range, rsi_upper_range)


Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [150]:
stats_train

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                    0.997009
Equity Final [$]                     3016.025
Equity Peak [$]                      3023.465
Return [%]                           0.534167
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                     0.13529
Volatility (Ann.) [%]                2.244895
Sharpe Ratio                         0.060265
Sortino Ratio                        0.105127
Calmar Ratio                         0.027767
Max. Drawdown [%]                   -4.872291
Avg. Drawdown [%]                   -4.872291
Max. Drawdown Duration      886 days 00:00:00
Avg. Drawdown Duration      886 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       3.313935
Worst Trade [%]                     -1.268585
Avg. Trade [%]                    

In [151]:
stats_val

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   45.309381
Equity Final [$]                     1614.122
Equity Peak [$]                        3000.0
Return [%]                         -46.195933
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                   -26.87617
Volatility (Ann.) [%]               10.249926
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -51.2685
Avg. Drawdown [%]                    -51.2685
Max. Drawdown Duration      707 days 00:00:00
Avg. Drawdown Duration      707 days 00:00:00
# Trades                                   48
Win Rate [%]                            31.25
Best Trade [%]                       3.903947
Worst Trade [%]                     -4.108963
Avg. Trade [%]                    

### Keltner Breakout

In [173]:
import talib
import numpy as np
from backtesting import Backtest, Strategy

# Keltner Channel Breakout Strategy
class KeltnerBreakout(Strategy):
    atr_period = 14  # Số phiên tính toán ATR
    risk_reward_ratio = 2  # Tỷ lệ R:R = 1:2
    stop_loss_pct = 0.02  # 2% Stop Loss
    take_profit_pct = 0.04  # 4% Take Profit

    def init(self):
        # Tính đường trung bình động và ATR
        self.middle_band = self.I(talib.EMA, self.data.Close, self.atr_period)
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period)
        
        # Tạo dải trên và dải dưới dựa trên ATR
        self.upper_band = self.middle_band + self.atr
        self.lower_band = self.middle_band - self.atr

    def next(self):
        # Mua khi giá vượt qua dải trên của Keltner Channel
        if self.data.Close[-1] > self.upper_band[-1]:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop Loss 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take Profit 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Bán khi giá vượt xuống dưới dải dưới của Keltner Channel
        elif self.data.Close[-1] < self.lower_band[-1]:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop Loss 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take Profit 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa Keltner Breakout
def run_backtest_optimize_keltner(train_data, val_data, period_range):
    # Backtest và tối ưu hóa trên tập train
    print(f"Optimizing KeltnerBreakout on training set...")
    bt_train = Backtest(train_data, KeltnerBreakout, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        atr_period=period_range,  # Phạm vi tối ưu atr_period
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, KeltnerBreakout, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
atr_period_range = range(10, 101, 5)  # Điều chỉnh khoảng giá trị atr_period
stats_train_keltner, stats_val_keltner = run_backtest_optimize_keltner(train_day, valid_day, atr_period_range)


Optimizing KeltnerBreakout on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/10 [00:00<?, ?it/s]

Backtesting on validation set...


In [174]:
stats_train_keltner

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   39.381854
Equity Final [$]                      653.418
Equity Peak [$]                        3000.0
Return [%]                           -78.2194
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                  -32.076865
Volatility (Ann.) [%]                16.38653
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -78.9407
Avg. Drawdown [%]                    -78.9407
Max. Drawdown Duration     1462 days 00:00:00
Avg. Drawdown Duration     1462 days 00:00:00
# Trades                                  162
Win Rate [%]                        31.481481
Best Trade [%]                       4.795394
Worst Trade [%]                     -7.641856
Avg. Trade [%]                    

In [175]:
stats_val_keltner

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   75.449102
Equity Final [$]                     1068.254
Equity Peak [$]                      3663.117
Return [%]                         -64.391533
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                  -40.635077
Volatility (Ann.) [%]               23.884744
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -71.422207
Avg. Drawdown [%]                  -16.110326
Max. Drawdown Duration      689 days 00:00:00
Avg. Drawdown Duration      142 days 00:00:00
# Trades                                  170
Win Rate [%]                        41.764706
Best Trade [%]                       4.004873
Worst Trade [%]                      -5.14495
Avg. Trade [%]                    

## Mean Reverse

### Looser Mean Reversion

In [10]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Mean Reversion Strategy
class MeanReversionLooser(Strategy):
    bb_period = 20  # Bollinger Bands period
    rsi_period = 14  # RSI period
    rsi_lower = 30  # RSI lower bound (oversold)
    rsi_upper = 70  # RSI upper bound (overbought)
    rsi_neutral = 50  # RSI trung lập
    risk_reward_ratio = 2  # Risk-Reward ratio
    stop_loss_pct = 0.02  # Stop loss là 2% dưới hoặc trên giá vào lệnh
    take_profit_pct = 0.04  # Take profit là 4% trên hoặc dưới giá vào lệnh

    def init(self):
        # Tính toán dải Bollinger Bands
        self.upper_band, self.middle_band, self.lower_band = self.I(
            talib.BBANDS, self.data.Close, timeperiod=self.bb_period, nbdevup=1.5, nbdevdn=1.5, matype=0)
        
        # Tính toán chỉ báo RSI
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period)

    def next(self):
        # Điều kiện Mua: Khi giá đóng cửa dưới dải dưới và RSI < rsi_lower
        if self.data.Close[-1] < self.lower_band[-1] and self.rsi[-1] < self.rsi_lower:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop loss là 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take profit là 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện Bán: Khi giá đóng cửa trên dải trên và RSI > rsi_upper
        elif self.data.Close[-1] > self.upper_band[-1] and self.rsi[-1] > self.rsi_upper:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop loss là 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take profit là 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)
        
        # Thoát lệnh khi giá quay lại đường giữa của Bollinger Band hoặc RSI quay về mức trung lập
        for trade in self.trades:
            if self.data.Close[-1] > self.middle_band[-1] and trade.is_long:
                trade.close()  # Thoát lệnh Mua nếu giá quay lại đường giữa
            elif self.data.Close[-1] < self.middle_band[-1] and trade.is_short:
                trade.close()  # Thoát lệnh Bán nếu giá quay lại đường giữa
            
            # Thoát lệnh khi RSI quay về mức trung lập
            if self.rsi[-1] > self.rsi_neutral and trade.is_long:
                trade.close()  # Thoát lệnh Mua khi RSI quay về mức trung lập
            elif self.rsi[-1] < self.rsi_neutral and trade.is_short:
                trade.close()  # Thoát lệnh Bán khi RSI quay về mức trung lập

# Hàm backtest và tối ưu hóa
def run_backtest_optimize(strategy_class, train_data, val_data, bb_period_range, rsi_period_range, rsi_lower_range, rsi_upper_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        bb_period=bb_period_range,  # Tối ưu bb_period
        rsi_period=rsi_period_range,  # Tối ưu rsi_period
        rsi_lower=rsi_lower_range,  # Tối ưu rsi_lower
        rsi_upper=rsi_upper_range,  # Tối ưu rsi_upper
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        constraint=lambda params: params.rsi_lower < params.rsi_upper,  # Điều kiện hợp lệ
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage
bb_period_range = range(10, 31, 5)  # Điều chỉnh khoảng giá trị bb_period
rsi_period_range = range(10, 21, 5)  # Điều chỉnh khoảng giá trị rsi_period
rsi_lower_range = range(20, 46, 5)  # Điều chỉnh khoảng giá trị rsi_lower
rsi_upper_range = range(55, 71, 5)  # Điều chỉnh khoảng giá trị rsi_upper

# Chạy backtest và tối ưu hóa cho chiến lược Mean Reversion
stats_train, stats_val = run_backtest_optimize(MeanReversionLooser, train_day, valid_day, bb_period_range, rsi_period_range, rsi_lower_range, rsi_upper_range)


Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 360 configurations.
  output = _optimize_grid()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting on validation set...


In [11]:
stats_train

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                    4.985045
Equity Final [$]                     1911.948
Equity Peak [$]                      3023.465
Return [%]                           -36.2684
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                  -10.803079
Volatility (Ann.) [%]                6.523346
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -41.388969
Avg. Drawdown [%]                  -41.388969
Max. Drawdown Duration      883 days 00:00:00
Avg. Drawdown Duration      883 days 00:00:00
# Trades                                   26
Win Rate [%]                        19.230769
Best Trade [%]                       3.313935
Worst Trade [%]                      -3.75725
Avg. Trade [%]                    

In [12]:
stats_val

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   26.546906
Equity Final [$]                     1530.967
Equity Peak [$]                        3000.0
Return [%]                         -48.967767
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                   -28.80351
Volatility (Ann.) [%]               13.019445
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -54.366867
Avg. Drawdown [%]                  -54.366867
Max. Drawdown Duration      700 days 00:00:00
Avg. Drawdown Duration      700 days 00:00:00
# Trades                                   57
Win Rate [%]                        31.578947
Best Trade [%]                       3.362682
Worst Trade [%]                     -5.276704
Avg. Trade [%]                    

### Multi Timeframe Mean Reversion

In [19]:
import talib
from backtesting import Backtest, Strategy

class MultiTimeframeMeanReversion(Strategy):
    ma_short = 20
    ma_long = 50

    def init(self):
        self.ma_short = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_short)
        self.ma_long = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_long)

    def next(self):
        if self.ma_short[-1] < self.ma_long[-1] and self.data.Close[-1] < self.ma_short[-1]:
            self.buy(size=1/6)

        elif self.ma_short[-1] > self.ma_long[-1] and self.data.Close[-1] > self.ma_short[-1]:
            self.sell(size=1/6)

# Hàm backtest và tối ưu hóa cho Multi-Timeframe Mean Reversion
def run_backtest_optimize_multi(train_data, val_data, ma_short_range, ma_long_range):
    print(f"Optimizing MultiTimeframeMeanReversion on training set...")
    bt_train = Backtest(train_data, MultiTimeframeMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        ma_short=ma_short_range,
        ma_long=ma_long_range,
        maximize='Return [%]',
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MultiTimeframeMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
ma_short_range = range(10, 31, 5)
ma_long_range = range(40, 101, 10)
stats_train_multi, stats_val_multi = run_backtest_optimize_multi(train_day, valid_day, ma_short_range, ma_long_range)


Optimizing MultiTimeframeMeanReversion on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [20]:
stats_train_multi

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   89.930209
Equity Final [$]                     7196.039
Equity Peak [$]                      8871.839
Return [%]                         139.867967
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                   24.861344
Volatility (Ann.) [%]              203.875779
Sharpe Ratio                         0.121944
Sortino Ratio                        0.442216
Calmar Ratio                         0.262944
Max. Drawdown [%]                  -94.550073
Avg. Drawdown [%]                   -6.209401
Max. Drawdown Duration      638 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   25
Win Rate [%]                             44.0
Best Trade [%]                      51.807328
Worst Trade [%]                    -43.995262
Avg. Trade [%]                    

In [21]:
stats_val_multi

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   89.820359
Equity Final [$]                     6353.938
Equity Peak [$]                      6386.938
Return [%]                         111.797933
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                      46.081
Volatility (Ann.) [%]               65.431953
Sharpe Ratio                         0.704258
Sortino Ratio                        1.943802
Calmar Ratio                         1.776348
Max. Drawdown [%]                  -25.941429
Avg. Drawdown [%]                   -9.014264
Max. Drawdown Duration      254 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                   18
Win Rate [%]                             50.0
Best Trade [%]                      49.971211
Worst Trade [%]                    -28.835159
Avg. Trade [%]                    

### Momentum Mean Reversion

In [244]:
import talib
from backtesting import Backtest, Strategy

# Momentum Mean Reversion Strategy
class MomentumMeanReversion(Strategy):
    rsi_period = 14  # Tham số mặc định cho RSI
    ma_period = 50  # Tham số mặc định cho MA dài hạn
    risk_reward_ratio = 2  # Tỷ lệ R:R = 2

    def init(self):
        self.ma = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_period)
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period)

    def next(self):
        price_diff = abs(self.data.Close[-1] - self.ma[-1])
        
        # Điều chỉnh SL và TP hợp lệ cho lệnh mua
        if self.rsi[-1] < 30 and self.data.Close[-1] < self.ma[-1]:
            sl = self.data.Close[-1] - price_diff  # Stop loss là mức giá thấp hơn giá hiện tại
            tp = self.data.Close[-1] + price_diff * self.risk_reward_ratio  # Take profit cao hơn giá hiện tại
            if sl < self.data.Close[-1] < tp:
                self.buy(size=1/6, sl=sl, tp=tp)

        # Điều chỉnh SL và TP hợp lệ cho lệnh bán
        elif self.rsi[-1] > 70 and self.data.Close[-1] > self.ma[-1]:
            sl = self.data.Close[-1] + price_diff  # Stop loss là mức giá cao hơn giá hiện tại
            tp = self.data.Close[-1] - price_diff * self.risk_reward_ratio  # Take profit thấp hơn giá hiện tại
            if tp < self.data.Close[-1] < sl:
                self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa Momentum Mean Reversion
def run_backtest_optimize_momentum(train_data, val_data, ma_period_range, rsi_period_range):
    print(f"Optimizing MomentumMeanReversion on training set...")
    bt_train = Backtest(train_data, MomentumMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        ma_period=ma_period_range,
        rsi_period=rsi_period_range,
        maximize='Return [%]',
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MomentumMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
ma_period_range = range(30, 61, 10)
rsi_period_range = range(10, 21, 5)
stats_train_momentum, stats_val_momentum = run_backtest_optimize_momentum(train_day, valid_day, ma_period_range, rsi_period_range)


Optimizing MomentumMeanReversion on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/12 [00:00<?, ?it/s]

Backtesting on validation set...


In [245]:
stats_train_momentum

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   69.890329
Equity Final [$]                     1019.355
Equity Peak [$]                   3483.829667
Return [%]                           -66.0215
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                  -23.962002
Volatility (Ann.) [%]               39.024168
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -76.682117
Avg. Drawdown [%]                  -23.056497
Max. Drawdown Duration      952 days 00:00:00
Avg. Drawdown Duration      282 days 00:00:00
# Trades                                   65
Win Rate [%]                        26.153846
Best Trade [%]                      55.932122
Worst Trade [%]                     -32.98758
Avg. Trade [%]                    

In [246]:
stats_val_momentum

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   80.239521
Equity Final [$]                     3102.092
Equity Peak [$]                      3755.692
Return [%]                           3.403067
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                    1.704349
Volatility (Ann.) [%]               40.333707
Sharpe Ratio                         0.042256
Sortino Ratio                        0.072767
Calmar Ratio                         0.028155
Max. Drawdown [%]                  -60.534597
Avg. Drawdown [%]                  -12.141563
Max. Drawdown Duration      546 days 00:00:00
Avg. Drawdown Duration       83 days 00:00:00
# Trades                                   30
Win Rate [%]                        33.333333
Best Trade [%]                      23.991445
Worst Trade [%]                    -10.118963
Avg. Trade [%]                    

## MACD

### Original MACD

In [212]:
import pandas as pd
from backtesting import Backtest, Strategy
import talib
from backtesting.lib import crossover

# Chiến lược MACD với R:R = 1/2
class MACDStrategy(Strategy):
    fastperiod = 12  # MACD Fast EMA period
    slowperiod = 26  # MACD Slow EMA period
    signalperiod = 9  # Signal period
    risk_reward_ratio = 0.5  # Tỷ lệ R:R = 1/2

    def init(self):
        # Tính MACD và Signal
        self.macd, self.signal, _ = self.I(talib.MACD, self.data.Close, fastperiod=self.fastperiod, 
                                           slowperiod=self.slowperiod, signalperiod=self.signalperiod)
    
    def next(self):
        price_diff = abs(self.data.Close[-1] - self.macd[-1])  # Khoảng cách giữa giá và MACD
        if price_diff > 0:
            sl = self.data.Close[-1] - price_diff  # Stop loss dựa trên MACD
            tp = self.data.Close[-1] + price_diff * self.risk_reward_ratio  # Take profit là một nửa khoảng cách này

            # Mua khi MACD cắt lên Signal và MACD > 0
            if crossover(self.macd, self.signal) and self.macd[-1] > 0:
                self.buy(size=1/6, sl=sl, tp=tp)

            # Bán khi MACD cắt xuống Signal và MACD < 0
            elif crossover(self.signal, self.macd) and self.macd[-1] < 0:
                sl = self.data.Close[-1] + price_diff
                tp = self.data.Close[-1] - price_diff * self.risk_reward_ratio
                self.sell(size=1/6, sl=sl, tp=tp)

# Hàm chạy backtest với tối ưu hóa cho MACD với R:R = 1/2
def run_backtest_optimize_macd(train_data, val_data, fastperiod_range, slowperiod_range, signalperiod_range):
    # Backtest và tối ưu hóa trên tập train
    print(f"Optimizing MACDStrategy on training set...")
    bt_train = Backtest(train_data, MACDStrategy, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        fastperiod=fastperiod_range,  # Phạm vi tối ưu fastperiod
        slowperiod=slowperiod_range,  # Phạm vi tối ưu slowperiod
        signalperiod=signalperiod_range,  # Phạm vi tối ưu signalperiod
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MACDStrategy, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
fastperiod_range = range(8, 13, 1)
slowperiod_range = range(20, 31, 2)
signalperiod_range = range(7, 12, 1)
stats_train_macd, stats_val_macd = run_backtest_optimize_macd(train_day, valid_day, fastperiod_range, slowperiod_range, signalperiod_range)


Optimizing MACDStrategy on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [213]:
stats_train_macd

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   93.419741
Equity Final [$]                  5831.828708
Equity Peak [$]                  11132.239708
Return [%]                           94.39429
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                   18.375294
Volatility (Ann.) [%]            52884.378644
Sharpe Ratio                         0.000347
Sortino Ratio                        0.245972
Calmar Ratio                         0.186241
Max. Drawdown [%]                  -98.664264
Avg. Drawdown [%]                  -10.235279
Max. Drawdown Duration      690 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                   21
Win Rate [%]                        76.190476
Best Trade [%]                      50.761871
Worst Trade [%]                    -49.622663
Avg. Trade [%]                    

In [214]:
stats_val_macd

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                    92.41517
Equity Final [$]                      407.798
Equity Peak [$]                       3257.24
Return [%]                         -86.406733
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                  -63.497732
Volatility (Ann.) [%]               67.674079
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -88.696013
Avg. Drawdown [%]                  -25.882255
Max. Drawdown Duration      654 days 00:00:00
Avg. Drawdown Duration      169 days 00:00:00
# Trades                                    3
Win Rate [%]                        33.333333
Best Trade [%]                      24.881079
Worst Trade [%]                    -53.000933
Avg. Trade [%]                    

### MACD with RSI

In [216]:
import pandas as pd
from backtesting import Backtest, Strategy
import talib
from backtesting.lib import crossover

# Chiến lược MACD với RSI và R:R = 1/2
class MACDWithRSIStrategy(Strategy):
    fastperiod = 12  # MACD Fast EMA period
    slowperiod = 26  # MACD Slow EMA period
    signalperiod = 9  # Signal period
    rsi_period = 14  # RSI period
    rsi_overbought = 70
    rsi_oversold = 30
    risk_reward_ratio = 0.5  # Tỷ lệ R:R = 1/2

    def init(self):
        # Tính MACD và Signal
        self.macd, self.signal, _ = self.I(talib.MACD, self.data.Close, fastperiod=self.fastperiod, 
                                           slowperiod=self.slowperiod, signalperiod=self.signalperiod)
        # Tính RSI để lọc tín hiệu
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period)

    def next(self):
        price_diff = abs(self.data.Close[-1] - self.macd[-1])  # Khoảng cách giữa giá và MACD
        if price_diff > 0:
            sl = self.data.Close[-1] - price_diff  # Stop loss dựa trên MACD
            tp = self.data.Close[-1] + price_diff * self.risk_reward_ratio  # Take profit là một nửa khoảng cách này

            # Mua khi MACD cắt lên Signal, MACD > 0 và RSI chưa vào vùng quá mua
            if crossover(self.macd, self.signal) and self.macd[-1] > 0 and self.rsi[-1] < self.rsi_overbought:
                self.buy(size=1/6, sl=sl, tp=tp)

            # Bán khi MACD cắt xuống Signal, MACD < 0 và RSI chưa vào vùng quá bán
            elif crossover(self.signal, self.macd) and self.macd[-1] < 0 and self.rsi[-1] > self.rsi_oversold:
                sl = self.data.Close[-1] + price_diff
                tp = self.data.Close[-1] - price_diff * self.risk_reward_ratio
                self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest với tối ưu hóa cho MACD với RSI
def run_backtest_optimize_macd_rsi(train_data, val_data, fastperiod_range, slowperiod_range, signalperiod_range):
    # Backtest và tối ưu hóa trên tập train
    print(f"Optimizing MACDWithRSIStrategy on training set...")
    bt_train = Backtest(train_data, MACDWithRSIStrategy, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        fastperiod=fastperiod_range,  # Phạm vi tối ưu fastperiod
        slowperiod=slowperiod_range,  # Phạm vi tối ưu slowperiod
        signalperiod=signalperiod_range,  # Phạm vi tối ưu signalperiod
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MACDWithRSIStrategy, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
fastperiod_range = range(8, 13, 1)
slowperiod_range = range(20, 31, 2)
signalperiod_range = range(7, 12, 1)
stats_train_macd_rsi, stats_val_macd_rsi = run_backtest_optimize_macd_rsi(train_day, valid_day, fastperiod_range, slowperiod_range, signalperiod_range)


Optimizing MACDWithRSIStrategy on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [217]:
stats_train_macd_rsi

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   93.419741
Equity Final [$]                  5850.583793
Equity Peak [$]                  11150.994793
Return [%]                           95.01946
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                   18.471789
Volatility (Ann.) [%]            52972.160666
Sharpe Ratio                         0.000349
Sortino Ratio                        0.247133
Calmar Ratio                         0.187219
Max. Drawdown [%]                  -98.664264
Avg. Drawdown [%]                  -10.317017
Max. Drawdown Duration      691 days 00:00:00
Avg. Drawdown Duration       47 days 00:00:00
# Trades                                   21
Win Rate [%]                        76.190476
Best Trade [%]                      50.761871
Worst Trade [%]                    -49.622663
Avg. Trade [%]                    

In [218]:
stats_val_macd_rsi

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                   89.421158
Equity Final [$]                     1389.219
Equity Peak [$]                        3000.0
Return [%]                           -53.6927
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                  -32.212499
Volatility (Ann.) [%]               21.905833
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -54.3527
Avg. Drawdown [%]                    -54.3527
Max. Drawdown Duration      654 days 00:00:00
Avg. Drawdown Duration      654 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                      24.881079
Worst Trade [%]                    -53.000933
Avg. Trade [%]                    

### MACD với ADX

In [219]:
import pandas as pd
from backtesting import Backtest, Strategy
import talib
from backtesting.lib import crossover

# Chiến lược MACD với ADX và R:R = 1/2
class MACDWithADXStrategy(Strategy):
    fastperiod = 12  # MACD Fast EMA period
    slowperiod = 26  # MACD Slow EMA period
    signalperiod = 9  # Signal period
    adx_period = 14  # ADX period
    adx_threshold = 25  # ADX threshold để xác định xu hướng mạnh
    risk_reward_ratio = 0.5  # Tỷ lệ R:R = 1/2

    def init(self):
        # Tính MACD và Signal
        self.macd, self.signal, _ = self.I(talib.MACD, self.data.Close, fastperiod=self.fastperiod, 
                                           slowperiod=self.slowperiod, signalperiod=self.signalperiod)
        # Tính ADX để xác định xu hướng mạnh
        self.adx = self.I(talib.ADX, self.data.High, self.data.Low, self.data.Close, timeperiod=self.adx_period)

    def next(self):
        price_diff = abs(self.data.Close[-1] - self.macd[-1])  # Khoảng cách giữa giá và MACD
        if price_diff > 0:
            sl = self.data.Close[-1] - price_diff  # Stop loss dựa trên MACD
            tp = self.data.Close[-1] + price_diff * self.risk_reward_ratio  # Take profit là một nửa khoảng cách này

            # Mua khi MACD cắt lên Signal, MACD > 0 và ADX > ngưỡng xu hướng mạnh
            if crossover(self.macd, self.signal) and self.macd[-1] > 0 and self.adx[-1] > self.adx_threshold:
                self.buy(size=1/6, sl=sl, tp=tp)

            # Bán khi MACD cắt xuống Signal, MACD < 0 và ADX > ngưỡng xu hướng mạnh
            elif crossover(self.signal, self.macd) and self.macd[-1] < 0 and self.adx[-1] > self.adx_threshold:
                sl = self.data.Close[-1] + price_diff
                tp = self.data.Close[-1] - price_diff * self.risk_reward_ratio
                self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest với tối ưu hóa cho MACD với ADX
def run_backtest_optimize_macd_adx(train_data, val_data, fastperiod_range, slowperiod_range, signalperiod_range):
    # Backtest và tối ưu hóa trên tập train
    print(f"Optimizing MACDWithADXStrategy on training set...")
    bt_train = Backtest(train_data, MACDWithADXStrategy, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        fastperiod=fastperiod_range,  # Phạm vi tối ưu fastperiod
        slowperiod=slowperiod_range,  # Phạm vi tối ưu slowperiod
        signalperiod=signalperiod_range,  # Phạm vi tối ưu signalperiod
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MACDWithADXStrategy, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
fastperiod_range = range(8, 13, 1)
slowperiod_range = range(20, 31, 2)
signalperiod_range = range(7, 12, 1)
stats_train_macd_adx, stats_val_macd_adx = run_backtest_optimize_macd_adx(train_day, valid_day, fastperiod_range, slowperiod_range, signalperiod_range)


Optimizing MACDWithADXStrategy on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [220]:
stats_train_macd_adx

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                    93.32004
Equity Final [$]                  3272.854595
Equity Peak [$]                   4262.294595
Return [%]                           9.095153
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    2.233713
Volatility (Ann.) [%]               58.843095
Sharpe Ratio                          0.03796
Sortino Ratio                        0.070701
Calmar Ratio                         0.044981
Max. Drawdown [%]                  -49.658762
Avg. Drawdown [%]                  -20.740798
Max. Drawdown Duration     1223 days 00:00:00
Avg. Drawdown Duration      230 days 00:00:00
# Trades                                   10
Win Rate [%]                             40.0
Best Trade [%]                      48.684382
Worst Trade [%]                    -55.931621
Avg. Trade [%]                    

In [221]:
stats_val_macd_adx

Start                     2022-08-31 09:00:00
End                       2024-08-30 09:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                    92.41517
Equity Final [$]                       143.13
Equity Peak [$]                       3257.24
Return [%]                            -95.229
Buy & Hold Return [%]                2.998686
Return (Ann.) [%]                  -78.487815
Volatility (Ann.) [%]              257.199145
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -97.024168
Avg. Drawdown [%]                  -27.964293
Max. Drawdown Duration      654 days 00:00:00
Avg. Drawdown Duration      169 days 00:00:00
# Trades                                    2
Win Rate [%]                              0.0
Best Trade [%]                     -38.474529
Worst Trade [%]                    -53.000933
Avg. Trade [%]                    

# 1-hour strategies

## Breakout

### Breakout with Bollinger Bands

In [346]:
import talib
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Bollinger Breakout Strategy
class BollingerBreakout(Strategy):
    bb_period = 20  # Bollinger Bands period
    risk_reward_ratio = 2  # Tỷ lệ R:R = 1:2
    stop_loss_pct = 0.02  # Stop Loss là 2% dưới giá vào lệnh
    take_profit_pct = 0.04  # Take Profit là 4% trên giá vào lệnh
    min_diff = 0.001  # Để đảm bảo có chênh lệch hợp lý trước khi đặt lệnh

    def init(self):
        # Sử dụng Bollinger Bands với các tham số mặc định
        self.upper_band, self.middle_band, self.lower_band = self.I(
            talib.BBANDS, self.data.Close, timeperiod=self.bb_period, nbdevup=1.5, nbdevdn=1.5, matype=0)

    def next(self):
        # Kiểm tra điều kiện crossover để vào lệnh
        if not self.position:  # Kiểm tra xem không có vị thế nào đang mở
            if crossover(self.data.Close, self.upper_band):  # Mua khi giá vượt qua dải trên
                sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # SL là 2% dưới giá hiện tại
                tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # TP là 4% trên giá hiện tại
                self.buy(size=1/6, sl=sl, tp=tp)
            
            elif crossover(self.lower_band, self.data.Close):  # Bán khi giá vượt xuống dưới dải dưới
                sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # SL là 2% trên giá hiện tại (cho lệnh bán)
                tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # TP là 4% dưới giá hiện tại (cho lệnh bán)
                self.sell(size=1/6, sl=sl, tp=tp)



# Hàm backtest với hàm optimize
def run_backtest_optimize(strategy_class, train_data, val_data, bb_period_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        bb_period=bb_period_range,  # Phạm vi tối ưu bb_period
        maximize='Return [%]',  # Tối ưu lợi nhuận Return [%]
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation với thông số tối ưu
    print("Backtesting on validation set...")
    optimized_bb_period = stats_train._strategy.bb_period
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run(bb_period=optimized_bb_period)
    
    return stats_train, stats_val

# Example: Chạy trên dữ liệu train_day và valid_day
bb_period_range = range(10, 101, 5)  # Điều chỉnh khoảng giá trị bb_period
stats_train, stats_val = run_backtest_optimize(BollingerBreakout, train_hour, valid_hour, bb_period_range)

Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/10 [00:00<?, ?it/s]

Backtesting on validation set...


In [347]:
stats_train

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   74.249354
Equity Final [$]                     3693.098
Equity Peak [$]                      5259.579
Return [%]                          23.103267
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                    5.339149
Volatility (Ann.) [%]               21.360342
Sharpe Ratio                         0.249956
Sortino Ratio                         0.40174
Calmar Ratio                         0.170142
Max. Drawdown [%]                  -31.380477
Avg. Drawdown [%]                   -3.256751
Max. Drawdown Duration      517 days 01:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                  104
Win Rate [%]                        52.884615
Best Trade [%]                       5.104139
Worst Trade [%]                     -3.745332
Avg. Trade [%]                    

In [348]:
stats_val 

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   73.826571
Equity Final [$]                     2798.877
Equity Peak [$]                      3419.534
Return [%]                            -6.7041
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                   -3.410197
Volatility (Ann.) [%]               18.019947
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -26.570433
Avg. Drawdown [%]                   -3.437023
Max. Drawdown Duration      689 days 01:00:00
Avg. Drawdown Duration       64 days 19:00:00
# Trades                                   59
Win Rate [%]                        49.152542
Best Trade [%]                       3.288722
Worst Trade [%]                     -3.852004
Avg. Trade [%]                    

### Breakout with Volume

In [349]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy

class VolumeBreakout(Strategy):
    risk_reward_ratio = 2  # Tỷ lệ R:R = 1:2
    volume_period = 20  # Số phiên trung bình khối lượng
    stop_loss_pct = 0.02  # Stop Loss là 2% dưới/ trên giá vào lệnh
    take_profit_pct = 0.04  # Take Profit là 4% trên/ dưới giá vào lệnh

    def init(self):
        # Chuyển đổi khối lượng thành kiểu float64 để phù hợp với talib
        volume_float = self.data.Volume.astype(np.float64)
        # Tính khối lượng trung bình
        self.volume_avg = self.I(talib.SMA, volume_float, timeperiod=self.volume_period)

    def next(self):
        # Kiểm tra khối lượng hiện tại có vượt khối lượng trung bình không
        if self.data.Volume[-1] > self.volume_avg[-1]:
            if not self.position:  # Kiểm tra xem có vị thế nào đang mở không
                # Lệnh mua
                if self.data.Close[-1] > self.data.Open[-1]:
                    sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # SL là 2% dưới giá hiện tại
                    tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # TP là 4% trên giá hiện tại
                    self.buy(size=1/6, sl=sl, tp=tp)

                # Lệnh bán
                elif self.data.Close[-1] < self.data.Open[-1]:
                    sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # SL là 2% trên giá hiện tại
                    tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # TP là 4% dưới giá hiện tại
                    self.sell(size=1/6, sl=sl, tp=tp)


# Hàm để thực hiện tối ưu hóa và backtest
def run_backtest_optimize(strategy_class, train_data, val_data, volume_period_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        volume_period=volume_period_range,  # Phạm vi tối ưu volume_period
        maximize='Return [%]',  # Tối ưu Sharpe Ratio
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage
volume_period_range = range(10, 101, 5)  # Điều chỉnh khoảng giá trị volume_period
stats_train, stats_val = run_backtest_optimize(VolumeBreakout, train_hour, valid_hour, volume_period_range)


Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/10 [00:00<?, ?it/s]

Backtesting on validation set...


In [350]:
stats_train

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   94.531716
Equity Final [$]                      1263.98
Equity Peak [$]                       3414.92
Return [%]                         -57.867333
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -19.450742
Volatility (Ann.) [%]               18.624058
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -62.98947
Avg. Drawdown [%]                    -6.06161
Max. Drawdown Duration     1276 days 21:00:00
Avg. Drawdown Duration       96 days 14:00:00
# Trades                                  170
Win Rate [%]                             40.0
Best Trade [%]                       5.743481
Worst Trade [%]                     -4.884552
Avg. Trade [%]                    

In [351]:
stats_val

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   96.062053
Equity Final [$]                     2327.627
Equity Peak [$]                      3457.852
Return [%]                         -22.412433
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -11.916195
Volatility (Ann.) [%]               21.244789
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -36.715018
Avg. Drawdown [%]                   -6.731836
Max. Drawdown Duration      669 days 03:00:00
Avg. Drawdown Duration       72 days 17:00:00
# Trades                                   97
Win Rate [%]                        46.391753
Best Trade [%]                       4.317095
Worst Trade [%]                     -3.723313
Avg. Trade [%]                    

### Breakout with RSI

In [355]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Chiến thuật Break-out với RSI có Stop Loss, Take Profit và kích thước 1/6
class RSI_Breakout(Strategy):
    rsi_period = 14
    rsi_lower = 35  # Điều chỉnh ngưỡng quá bán để tăng số lượng giao dịch
    rsi_upper = 65  # Điều chỉnh ngưỡng quá mua để tăng số lượng giao dịch
    stop_loss_pct = 0.02  # Stop Loss 2%
    take_profit_pct = 0.04  # Take Profit 4%

    def init(self):
        # Tính chỉ báo RSI
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Mua khi RSI dưới ngưỡng quá bán
        if not self.position and self.rsi[-1] < self.rsi_lower:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)
            self.buy(size=1/6, sl=sl, tp=tp)  # Mở lệnh mua với Stop Loss và Take Profit

        # Bán khi RSI vượt ngưỡng quá mua
        elif not self.position and self.rsi[-1] > self.rsi_upper:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)
            self.sell(size=1/6, sl=sl, tp=tp)  # Mở lệnh bán với Stop Loss và Take Profit


# Hàm để thực hiện backtest và tối ưu hóa
def run_backtest_optimize(strategy_class, train_data, val_data, rsi_period_range, rsi_lower_range, rsi_upper_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        rsi_period=rsi_period_range,  # Phạm vi tối ưu rsi_period
        rsi_lower=rsi_lower_range,  # Phạm vi tối ưu rsi_lower
        rsi_upper=rsi_upper_range,  # Phạm vi tối ưu rsi_upper
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        constraint=lambda params: params.rsi_lower < params.rsi_upper,  # Điều kiện hợp lệ cho RSI
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage
rsi_period_range = range(10, 31, 5)  # Điều chỉnh khoảng giá trị rsi_period
rsi_lower_range = range(20, 46, 5)  # Điều chỉnh khoảng giá trị rsi_lower
rsi_upper_range = range(60, 86, 5)  # Điều chỉnh khoảng giá trị rsi_upper

# Thực hiện backtest và tối ưu hóa
stats_train, stats_val = run_backtest_optimize(RSI_Breakout,  train_hour, valid_hour, rsi_period_range, rsi_lower_range, rsi_upper_range)


Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [356]:
stats_train

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                    1.391927
Equity Final [$]                     2839.425
Equity Peak [$]                      3094.916
Return [%]                            -5.3525
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                   -1.367201
Volatility (Ann.) [%]                3.873802
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.980002
Avg. Drawdown [%]                   -5.304164
Max. Drawdown Duration     1408 days 02:00:00
Avg. Drawdown Duration      471 days 01:00:00
# Trades                                    3
Win Rate [%]                        33.333333
Best Trade [%]                       3.001094
Worst Trade [%]                     -4.090486
Avg. Trade [%]                    

In [357]:
stats_val

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   71.599045
Equity Final [$]                       950.58
Equity Peak [$]                        3000.0
Return [%]                            -68.314
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -43.709681
Volatility (Ann.) [%]               10.056975
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                     -68.314
Avg. Drawdown [%]                     -68.314
Max. Drawdown Duration      723 days 00:00:00
Avg. Drawdown Duration      723 days 00:00:00
# Trades                                   82
Win Rate [%]                        26.829268
Best Trade [%]                       3.903947
Worst Trade [%]                     -3.920116
Avg. Trade [%]                    

### Keltner Breakout

In [358]:
import talib
import numpy as np
from backtesting import Backtest, Strategy

# Keltner Channel Breakout Strategy
class KeltnerBreakout(Strategy):
    atr_period = 14  # Số phiên tính toán ATR
    risk_reward_ratio = 2  # Tỷ lệ R:R = 1:2
    stop_loss_pct = 0.02  # 2% Stop Loss
    take_profit_pct = 0.04  # 4% Take Profit

    def init(self):
        # Tính đường trung bình động và ATR
        self.middle_band = self.I(talib.EMA, self.data.Close, self.atr_period)
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period)
        
        # Tạo dải trên và dải dưới dựa trên ATR
        self.upper_band = self.middle_band + self.atr
        self.lower_band = self.middle_band - self.atr

    def next(self):
        # Mua khi giá vượt qua dải trên của Keltner Channel
        if self.data.Close[-1] > self.upper_band[-1]:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop Loss 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take Profit 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Bán khi giá vượt xuống dưới dải dưới của Keltner Channel
        elif self.data.Close[-1] < self.lower_band[-1]:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop Loss 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take Profit 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa Keltner Breakout
def run_backtest_optimize_keltner(train_data, val_data, period_range):
    # Backtest và tối ưu hóa trên tập train
    print(f"Optimizing KeltnerBreakout on training set...")
    bt_train = Backtest(train_data, KeltnerBreakout, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        atr_period=period_range,  # Phạm vi tối ưu atr_period
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, KeltnerBreakout, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
atr_period_range = range(10, 101, 5)  # Điều chỉnh khoảng giá trị atr_period
stats_train_keltner, stats_val_keltner = run_backtest_optimize_keltner( train_hour, valid_hour, atr_period_range)


Optimizing KeltnerBreakout on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/10 [00:00<?, ?it/s]

Backtesting on validation set...


In [359]:
stats_train_keltner

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   49.353748
Equity Final [$]                      630.338
Equity Peak [$]                        3000.0
Return [%]                         -78.988733
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -32.322416
Volatility (Ann.) [%]               22.542486
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -79.687533
Avg. Drawdown [%]                  -79.687533
Max. Drawdown Duration     1470 days 01:00:00
Avg. Drawdown Duration     1470 days 01:00:00
# Trades                                  246
Win Rate [%]                         34.95935
Best Trade [%]                       6.024029
Worst Trade [%]                     -5.902228
Avg. Trade [%]                    

In [360]:
stats_val_keltner

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   84.208433
Equity Final [$]                      920.984
Equity Peak [$]                      6456.321
Return [%]                         -69.300533
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -44.592901
Volatility (Ann.) [%]               37.362837
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -85.735158
Avg. Drawdown [%]                  -10.781608
Max. Drawdown Duration      689 days 03:00:00
Avg. Drawdown Duration       51 days 09:00:00
# Trades                                  357
Win Rate [%]                        47.058824
Best Trade [%]                       4.317095
Worst Trade [%]                     -5.220411
Avg. Trade [%]                    

## Mean Reverse

### Looser Mean Reversion

In [329]:
import talib
import numpy as np
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Mean Reversion Strategy
class MeanReversionLooser(Strategy):
    bb_period = 20  # Bollinger Bands period
    rsi_period = 14  # RSI period
    rsi_lower = 30  # RSI lower bound (oversold)
    rsi_upper = 70  # RSI upper bound (overbought)
    risk_reward_ratio = 2  # Risk-Reward ratio
    stop_loss_pct = 0.02  # Stop loss là 2% dưới hoặc trên giá vào lệnh
    take_profit_pct = 0.04  # Take profit là 4% trên hoặc dưới giá vào lệnh

    def init(self):
        # Tính toán dải Bollinger Bands
        self.upper_band, self.middle_band, self.lower_band = self.I(
            talib.BBANDS, self.data.Close, timeperiod=self.bb_period, nbdevup=1.5, nbdevdn=1.5, matype=0)
        
        # Tính toán chỉ báo RSI
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period)

    def next(self):
        # Điều kiện Mua: Khi giá đóng cửa dưới dải dưới và RSI < rsi_lower
        if self.data.Close[-1] < self.lower_band[-1] and self.rsi[-1] < self.rsi_lower:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop loss là 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take profit là 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện Bán: Khi giá đóng cửa trên dải trên và RSI > rsi_upper
        elif self.data.Close[-1] > self.upper_band[-1] and self.rsi[-1] > self.rsi_upper:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop loss là 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take profit là 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)


# Hàm backtest và tối ưu hóa
def run_backtest_optimize(strategy_class, train_data, val_data, bb_period_range, rsi_period_range, rsi_lower_range, rsi_upper_range):
    # Backtest và tối ưu hóa trên tập train
    print("Optimizing on training set...")
    bt_train = Backtest(train_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_train = bt_train.optimize(
        bb_period=bb_period_range,  # Tối ưu bb_period
        rsi_period=rsi_period_range,  # Tối ưu rsi_period
        rsi_lower=rsi_lower_range,  # Tối ưu rsi_lower
        rsi_upper=rsi_upper_range,  # Tối ưu rsi_upper
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        constraint=lambda params: params.rsi_lower < params.rsi_upper,  # Điều kiện hợp lệ
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, strategy_class, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage
bb_period_range = range(10, 31, 5)  # Điều chỉnh khoảng giá trị bb_period
rsi_period_range = range(10, 21, 5)  # Điều chỉnh khoảng giá trị rsi_period
rsi_lower_range = range(20, 46, 5)  # Điều chỉnh khoảng giá trị rsi_lower
rsi_upper_range = range(55, 71, 5)  # Điều chỉnh khoảng giá trị rsi_upper

# Chạy backtest và tối ưu hóa cho chiến lược Mean Reversion
stats_train, stats_val = run_backtest_optimize(MeanReversionLooser, train_hour, valid_hour, bb_period_range, rsi_period_range, rsi_lower_range, rsi_upper_range)


Optimizing on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 360 configurations.
  output = _optimize_grid()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting on validation set...


In [335]:
stats_train

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   27.778882
Equity Final [$]                      967.965
Equity Peak [$]                      3230.802
Return [%]                           -67.7345
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -24.653577
Volatility (Ann.) [%]                17.25353
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -70.039482
Avg. Drawdown [%]                  -18.899696
Max. Drawdown Duration     1414 days 02:00:00
Avg. Drawdown Duration      358 days 12:00:00
# Trades                                  106
Win Rate [%]                        33.018868
Best Trade [%]                       3.359915
Worst Trade [%]                     -5.825696
Avg. Trade [%]                    

In [336]:
stats_val

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   30.111376
Equity Final [$]                      824.629
Equity Peak [$]                        3000.0
Return [%]                         -72.512367
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                   -47.57135
Volatility (Ann.) [%]               13.813552
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -72.512367
Avg. Drawdown [%]                  -72.512367
Max. Drawdown Duration      716 days 05:00:00
Avg. Drawdown Duration      716 days 05:00:00
# Trades                                   91
Win Rate [%]                         19.78022
Best Trade [%]                       3.189383
Worst Trade [%]                     -4.108963
Avg. Trade [%]                    

### Multi Timeframe Mean Reversion

In [364]:
import talib
from backtesting import Backtest, Strategy

class MultiTimeframeMeanReversion(Strategy):
    ma_short = 20  # Đường MA ngắn
    ma_long = 50  # Đường MA dài
    risk_reward_ratio = 0.5  # Tỷ lệ R:R = 1/2
    stop_loss_pct = 0.02  # Stop loss là 2% dưới hoặc trên giá vào lệnh
    take_profit_pct = 0.04  # Take profit là 4% trên hoặc dưới giá vào lệnh

    def init(self):
        # Tính toán các đường MA
        self.ma_short = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_short)
        self.ma_long = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_long)

    def next(self):
        # Điều kiện mua khi MA ngắn dưới MA dài và giá đóng cửa dưới MA ngắn
        if self.ma_short[-1] < self.ma_long[-1] and self.data.Close[-1] < self.ma_short[-1]:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop loss là 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take profit là 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán khi MA ngắn trên MA dài và giá đóng cửa trên MA ngắn
        elif self.ma_short[-1] > self.ma_long[-1] and self.data.Close[-1] > self.ma_short[-1]:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop loss là 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take profit là 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho Multi-Timeframe Mean Reversion
def run_backtest_optimize_multi(train_data, val_data, ma_short_range, ma_long_range):
    print(f"Optimizing MultiTimeframeMeanReversion on training set...")
    bt_train = Backtest(train_data, MultiTimeframeMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        ma_short=ma_short_range,
        ma_long=ma_long_range,
        maximize='Return [%]',
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MultiTimeframeMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
ma_short_range = range(10, 31, 5)
ma_long_range = range(40, 101, 10)
stats_train_multi, stats_val_multi = run_backtest_optimize_multi( train_hour, valid_hour, ma_short_range, ma_long_range)


Optimizing MultiTimeframeMeanReversion on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [365]:
stats_train_multi

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   38.775104
Equity Final [$]                      494.288
Equity Peak [$]                      3484.948
Return [%]                         -83.523733
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -36.317474
Volatility (Ann.) [%]               22.028499
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -86.823964
Avg. Drawdown [%]                  -18.257265
Max. Drawdown Duration     1414 days 02:00:00
Avg. Drawdown Duration      242 days 17:00:00
# Trades                                  248
Win Rate [%]                        38.306452
Best Trade [%]                       3.726386
Worst Trade [%]                     -6.208026
Avg. Trade [%]                    

In [366]:
stats_val_multi

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                    8.035004
Equity Final [$]                      732.731
Equity Peak [$]                        3000.0
Return [%]                         -75.575633
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -50.578986
Volatility (Ann.) [%]               14.999799
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -75.5853
Avg. Drawdown [%]                    -75.5853
Max. Drawdown Duration      717 days 00:00:00
Avg. Drawdown Duration      717 days 00:00:00
# Trades                                   98
Win Rate [%]                        18.367347
Best Trade [%]                       3.360808
Worst Trade [%]                     -5.276704
Avg. Trade [%]                    

### Momentum Mean Reversion

In [367]:
import talib
from backtesting import Backtest, Strategy

# Momentum Mean Reversion Strategy
class MomentumMeanReversion(Strategy):
    rsi_period = 14  # Tham số mặc định cho RSI
    ma_period = 50  # Tham số mặc định cho MA dài hạn
    stop_loss_pct = 0.02  # Stop Loss là 2% dưới/ trên giá vào lệnh
    take_profit_pct = 0.04  # Take Profit là 4% trên/ dưới giá vào lệnh

    def init(self):
        # Tính toán đường MA và RSI
        self.ma = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_period)
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period)

    def next(self):
        # Điều kiện mua: RSI dưới 30 và giá đóng cửa dưới MA
        if self.rsi[-1] < 30 and self.data.Close[-1] < self.ma[-1]:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop loss là 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take profit là 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán: RSI trên 70 và giá đóng cửa trên MA
        elif self.rsi[-1] > 70 and self.data.Close[-1] > self.ma[-1]:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop loss là 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take profit là 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa Momentum Mean Reversion
def run_backtest_optimize_momentum(train_data, val_data, ma_period_range, rsi_period_range):
    print(f"Optimizing MomentumMeanReversion on training set...")
    bt_train = Backtest(train_data, MomentumMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        ma_period=ma_period_range,
        rsi_period=rsi_period_range,
        maximize='Return [%]',
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MomentumMeanReversion, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
ma_period_range = range(30, 61, 10)
rsi_period_range = range(10, 21, 5)
stats_train_momentum, stats_val_momentum = run_backtest_optimize_momentum( train_hour, valid_hour, ma_period_range, rsi_period_range)


Optimizing MomentumMeanReversion on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/12 [00:00<?, ?it/s]

Backtesting on validation set...


In [368]:
stats_train_momentum

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   40.962418
Equity Final [$]                      743.274
Equity Peak [$]                      3941.951
Return [%]                           -75.2242
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -29.472826
Volatility (Ann.) [%]               24.798238
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -81.144514
Avg. Drawdown [%]                  -11.968562
Max. Drawdown Duration     1414 days 02:00:00
Avg. Drawdown Duration      160 days 17:00:00
# Trades                                  231
Win Rate [%]                        41.558442
Best Trade [%]                       3.359915
Worst Trade [%]                     -6.425278
Avg. Trade [%]                    

In [369]:
stats_val_momentum

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   25.019889
Equity Final [$]                      851.038
Equity Peak [$]                        3000.0
Return [%]                         -71.632067
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -46.738444
Volatility (Ann.) [%]               14.323507
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -73.025267
Avg. Drawdown [%]                  -73.025267
Max. Drawdown Duration      716 days 05:00:00
Avg. Drawdown Duration      716 days 05:00:00
# Trades                                   86
Win Rate [%]                        18.604651
Best Trade [%]                       3.189383
Worst Trade [%]                     -5.276704
Avg. Trade [%]                    

## Momentum Trading

### Momentum trading with MACD

In [374]:
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Chiến lược Momentum Trading với MACD
class MomentumTradingMACD(Strategy):
    fastperiod = 12  # MACD Fast EMA period
    slowperiod = 26  # MACD Slow EMA period
    signalperiod = 9  # Signal period
    risk_reward_ratio = 2  # Tỷ lệ R:R = 2
    stop_loss_pct = 0.02  # Stop loss 2%
    take_profit_pct = 0.04  # Take profit 4%

    def init(self):
        # Tính MACD và Signal
        self.macd, self.signal, _ = self.I(talib.MACD, self.data.Close, fastperiod=self.fastperiod, 
                                           slowperiod=self.slowperiod, signalperiod=self.signalperiod)

    def next(self):
        # Điều kiện mua: MACD cắt lên Signal và MACD > 0
        if crossover(self.macd, self.signal) and self.macd[-1] > 0:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop loss là 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take profit là 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán: MACD cắt xuống Signal và MACD < 0
        elif crossover(self.signal, self.macd) and self.macd[-1] < 0:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop loss là 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take profit là 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho MACD
def run_backtest_optimize_macd(train_data, val_data, fastperiod_range, slowperiod_range, signalperiod_range):
    bt_train = Backtest(train_data, MomentumTradingMACD, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        fastperiod=fastperiod_range,
        slowperiod=slowperiod_range,
        signalperiod=signalperiod_range,
        maximize='Return [%]',
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    bt_val = Backtest(val_data, MomentumTradingMACD, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Ví dụ cho từng chiến lược với các khoảng thời gian khác nhau
fastperiod_range = range(8, 13, 1)
slowperiod_range = range(20, 31, 2)
signalperiod_range = range(7, 12, 1)


# Backtest và tối ưu hóa cho MACD
stats_train_macd, stats_val_macd = run_backtest_optimize_macd(train_hour, valid_hour, fastperiod_range, slowperiod_range, signalperiod_range)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

In [375]:
stats_train_macd

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   60.588586
Equity Final [$]                     1611.981
Equity Peak [$]                        3000.0
Return [%]                           -46.2673
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -14.396269
Volatility (Ann.) [%]                16.82589
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -59.185467
Avg. Drawdown [%]                  -59.185467
Max. Drawdown Duration     1459 days 01:00:00
Avg. Drawdown Duration     1459 days 01:00:00
# Trades                                  149
Win Rate [%]                        43.624161
Best Trade [%]                       4.531698
Worst Trade [%]                     -4.115325
Avg. Trade [%]                    

In [376]:
stats_val_macd

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   59.427208
Equity Final [$]                     2266.566
Equity Peak [$]                      3841.018
Return [%]                           -24.4478
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -13.079231
Volatility (Ann.) [%]               19.546939
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -50.368496
Avg. Drawdown [%]                   -5.566982
Max. Drawdown Duration      549 days 00:00:00
Avg. Drawdown Duration       42 days 09:00:00
# Trades                                   72
Win Rate [%]                        45.833333
Best Trade [%]                       3.775424
Worst Trade [%]                      -3.48201
Avg. Trade [%]                    

### Momentum trading with Stochastic Oscillator

In [378]:
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Chiến lược Momentum Trading với Stochastic Oscillator
class MomentumTradingStochastic(Strategy):
    slowk_period = 14
    slowd_period = 3
    stop_loss_pct = 0.02
    take_profit_pct = 0.04
    risk_reward_ratio = 2  # Tỷ lệ R:R = 2

    def init(self):
        self.slowk, self.slowd = self.I(talib.STOCH, self.data.High, self.data.Low, self.data.Close,
                                        fastk_period=self.slowk_period, slowk_period=self.slowk_period,
                                        slowk_matype=0, slowd_period=self.slowd_period, slowd_matype=0)

    def next(self):
        # Mua khi %K cắt lên %D và %K < 20 (quá bán)
        if crossover(self.slowk, self.slowd) and self.slowk[-1] < 20:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)
            self.buy(size=1/6, sl=sl, tp=tp)

        # Bán khi %K cắt xuống %D và %K > 80 (quá mua)
        elif crossover(self.slowd, self.slowk) and self.slowk[-1] > 80:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho Stochastic Oscillator
def run_backtest_optimize_stochastic(train_data, val_data, slowk_range, slowd_range):
    bt_train = Backtest(train_data, MomentumTradingStochastic, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        slowk_period=slowk_range,
        slowd_period=slowd_range,
        maximize='Return [%]',
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    bt_val = Backtest(val_data, MomentumTradingStochastic, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val


slowk_range = range(10, 21, 5)
slowd_range = range(3, 10, 1)
# Backtest và tối ưu hóa cho Stochastic Oscillator
stats_train_stochastic, stats_val_stochastic = run_backtest_optimize_stochastic(train_hour, valid_hour, slowk_range, slowd_range)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/11 [00:00<?, ?it/s]

In [379]:
stats_train_stochastic

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   52.356333
Equity Final [$]                      1485.45
Equity Peak [$]                      3093.812
Return [%]                            -50.485
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -16.129659
Volatility (Ann.) [%]                 12.8367
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -53.829321
Avg. Drawdown [%]                  -11.289072
Max. Drawdown Duration     1448 days 00:00:00
Avg. Drawdown Duration      291 days 06:00:00
# Trades                                   88
Win Rate [%]                        35.227273
Best Trade [%]                       3.725335
Worst Trade [%]                     -3.706651
Avg. Trade [%]                    

In [380]:
stats_val_stochastic

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   64.319809
Equity Final [$]                     1160.419
Equity Peak [$]                        3000.0
Return [%]                         -61.319367
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -37.806244
Volatility (Ann.) [%]               11.794888
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -61.319367
Avg. Drawdown [%]                  -61.319367
Max. Drawdown Duration      721 days 00:00:00
Avg. Drawdown Duration      721 days 00:00:00
# Trades                                   85
Win Rate [%]                        30.588235
Best Trade [%]                       3.583092
Worst Trade [%]                     -3.877888
Avg. Trade [%]                    

# 3-minute

## Scalping

### Scalping with RSI and Stochastic Oscillator

In [386]:
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Chiến lược Scalping với RSI và Stochastic Oscillator
class ScalpingRSIStochastic(Strategy):
    rsi_period = 14
    stoch_k_period = 14
    stoch_d_period = 3
    rsi_overbought = 70
    rsi_oversold = 30
    risk_reward_ratio = 2  # Tỷ lệ R:R = 2
    stop_loss_pct = 0.02  # Stop loss 2%
    take_profit_pct = 0.04  # Take profit 4%

    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period)
        self.stoch_k, self.stoch_d = self.I(talib.STOCH, self.data.High, self.data.Low, self.data.Close, 
                                            fastk_period=self.stoch_k_period, slowk_period=self.stoch_d_period, 
                                            slowk_matype=0, slowd_period=self.stoch_d_period, slowd_matype=0)

    def next(self):
        # Điều kiện mua: RSI < 30 và Stochastic < 20 (quá bán)
        if self.rsi[-1] < self.rsi_oversold and self.stoch_k[-1] < 20 and self.stoch_d[-1] < 20:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán: RSI > 70 và Stochastic > 80 (quá mua)
        elif self.rsi[-1] > self.rsi_overbought and self.stoch_k[-1] > 80 and self.stoch_d[-1] > 80:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho Scalping
def run_backtest_optimize_scalping(train_data, val_data, rsi_period_range, stoch_k_period_range, stoch_d_period_range):
    print(f"Optimizing ScalpingRSIStochastic on training set...")
    bt_train = Backtest(train_data, ScalpingRSIStochastic, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        rsi_period=rsi_period_range,  # Phạm vi tối ưu RSI period
        stoch_k_period=stoch_k_period_range,  # Phạm vi tối ưu Stochastic K period
        stoch_d_period=stoch_d_period_range,  # Phạm vi tối ưu Stochastic D period
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, ScalpingRSIStochastic, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
rsi_period_range = range(10, 21, 5)
stoch_k_period_range = range(10, 21, 5)
stoch_d_period_range = range(3, 8, 2)

stats_train_scalping, stats_val_scalping = run_backtest_optimize_scalping(train_min, valid_min, rsi_period_range, stoch_k_period_range, stoch_d_period_range)


Optimizing ScalpingRSIStochastic on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [387]:
stats_train_scalping

Start                     2018-08-13 09:00:00
End                       2022-08-26 09:12:00
Duration                   1474 days 00:12:00
Exposure Time [%]                   40.106382
Equity Final [$]                      503.263
Equity Peak [$]                      3092.643
Return [%]                         -83.224567
Buy & Hold Return [%]               38.229995
Return (Ann.) [%]                  -35.211253
Volatility (Ann.) [%]               21.599161
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -83.72709
Avg. Drawdown [%]                  -13.859684
Max. Drawdown Duration     1470 days 21:57:00
Avg. Drawdown Duration      210 days 12:57:00
# Trades                                  293
Win Rate [%]                        37.542662
Best Trade [%]                       4.181601
Worst Trade [%]                     -5.825696
Avg. Trade [%]                    

In [388]:
stats_val_scalping

Start                     2022-08-26 09:15:00
End                       2024-08-30 14:45:00
Duration                    735 days 05:30:00
Exposure Time [%]                   11.246885
Equity Final [$]                      703.464
Equity Peak [$]                        3000.0
Return [%]                           -76.5512
Buy & Hold Return [%]                2.177413
Return (Ann.) [%]                  -49.942485
Volatility (Ann.) [%]               14.623344
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -76.5512
Avg. Drawdown [%]                    -76.5512
Max. Drawdown Duration      735 days 03:48:00
Avg. Drawdown Duration      735 days 03:48:00
# Trades                                  118
Win Rate [%]                        24.576271
Best Trade [%]                       4.470006
Worst Trade [%]                     -4.229546
Avg. Trade [%]                    

### Scalping with EMA Crossovers

In [401]:
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Chiến lược Scalping với EMA Crossovers
class ScalpingEMA(Strategy):
    fast_ema_period = 9  # EMA ngắn hạn
    slow_ema_period = 21  # EMA dài hạn
    stop_loss_pct = 0.02  # Stop loss 2%
    take_profit_pct = 0.04  # Take profit 4%

    def init(self):
        # Tính EMA
        self.fast_ema = self.I(talib.EMA, self.data.Close, timeperiod=self.fast_ema_period)
        self.slow_ema = self.I(talib.EMA, self.data.Close, timeperiod=self.slow_ema_period)

    def next(self):
        # Điều kiện mua: EMA ngắn cắt lên EMA dài
        if crossover(self.fast_ema, self.slow_ema):
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop loss 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take profit 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán: EMA ngắn cắt xuống EMA dài
        elif crossover(self.slow_ema, self.fast_ema):
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop loss 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take profit 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho Scalping với EMA
def run_backtest_optimize_scalping_ema(train_data, val_data, fast_ema_range, slow_ema_range):
    print(f"Optimizing ScalpingEMA on training set...")
    bt_train = Backtest(train_data, ScalpingEMA, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        fast_ema_period=fast_ema_range,  # Phạm vi tối ưu EMA ngắn
        slow_ema_period=slow_ema_range,  # Phạm vi tối ưu EMA dài
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, ScalpingEMA, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
fast_ema_range = range(5, 11, 1)
slow_ema_range = range(20, 31, 2)
stats_train_ema, stats_val_ema = run_backtest_optimize_scalping_ema(train_min, valid_min, fast_ema_range, slow_ema_range)


Optimizing ScalpingEMA on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [402]:
stats_train_ema

Start                     2018-08-13 09:00:00
End                       2022-08-26 09:12:00
Duration                   1474 days 00:12:00
Exposure Time [%]                   48.185641
Equity Final [$]                      615.163
Equity Peak [$]                        3000.0
Return [%]                         -79.494567
Buy & Hold Return [%]               38.229995
Return (Ann.) [%]                  -32.238403
Volatility (Ann.) [%]                13.01164
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -79.940433
Avg. Drawdown [%]                  -79.940433
Max. Drawdown Duration     1473 days 22:30:00
Avg. Drawdown Duration     1473 days 22:30:00
# Trades                                  326
Win Rate [%]                        37.423313
Best Trade [%]                       6.733087
Worst Trade [%]                      -6.70137
Avg. Trade [%]                    

### Scalping with Bollinger Bands

In [406]:
import talib
from backtesting import Strategy, Backtest

# Chiến lược Scalping với Bollinger Bands
class ScalpingBollingerBands(Strategy):
    bb_period = 20  # Chu kỳ Bollinger Bands
    stop_loss_pct = 0.02  # Stop loss 2%
    take_profit_pct = 0.04  # Take profit 4%

    def init(self):
        # Tính Bollinger Bands
        self.upper_band, self.middle_band, self.lower_band = self.I(talib.BBANDS, self.data.Close, timeperiod=self.bb_period)

    def next(self):
        # Mua khi giá dưới Bollinger Band dưới
        if self.data.Close[-1] < self.lower_band[-1]:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)
            self.buy(size=1/6, sl=sl, tp=tp)

        # Bán khi giá trên Bollinger Band trên
        elif self.data.Close[-1] > self.upper_band[-1]:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho Scalping với Bollinger Bands
def run_backtest_optimize_scalping_bb(train_data, val_data, bb_period_range):
    print(f"Optimizing ScalpingBollingerBands on training set...")
    bt_train = Backtest(train_data, ScalpingBollingerBands, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        bb_period=bb_period_range,  # Phạm vi tối ưu bb_period
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, ScalpingBollingerBands, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
bb_period_range = range(10, 31, 5)
stats_train_bb, stats_val_bb = run_backtest_optimize_scalping_bb(train_min, valid_min, bb_period_range)


Optimizing ScalpingBollingerBands on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/5 [00:00<?, ?it/s]

Backtesting on validation set...


In [407]:
stats_train_bb

Start                     2018-08-13 09:00:00
End                       2022-08-26 09:12:00
Duration                   1474 days 00:12:00
Exposure Time [%]                   36.278797
Equity Final [$]                      460.795
Equity Peak [$]                        3000.0
Return [%]                         -84.640167
Buy & Hold Return [%]               38.229995
Return (Ann.) [%]                  -36.532235
Volatility (Ann.) [%]                13.50114
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -84.640167
Avg. Drawdown [%]                  -84.640167
Max. Drawdown Duration     1473 days 22:27:00
Avg. Drawdown Duration     1473 days 22:27:00
# Trades                                  218
Win Rate [%]                        30.275229
Best Trade [%]                       6.030307
Worst Trade [%]                     -5.354141
Avg. Trade [%]                    

In [408]:
stats_val_bb

Start                     2022-08-26 09:15:00
End                       2024-08-30 14:45:00
Duration                    735 days 05:30:00
Exposure Time [%]                   11.723692
Equity Final [$]                      692.639
Equity Peak [$]                        3000.0
Return [%]                         -76.912033
Buy & Hold Return [%]                2.177413
Return (Ann.) [%]                  -50.222716
Volatility (Ann.) [%]               15.564928
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -76.912033
Avg. Drawdown [%]                  -76.912033
Max. Drawdown Duration      735 days 04:15:00
Avg. Drawdown Duration      735 days 04:15:00
# Trades                                  105
Win Rate [%]                        20.952381
Best Trade [%]                       4.470006
Worst Trade [%]                     -4.131413
Avg. Trade [%]                    

## Momentum trading

### Momentum trading with MACD

In [389]:
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Chiến lược Momentum Trading với MACD
class MomentumTradingMACD(Strategy):
    fastperiod = 12  # MACD Fast EMA period
    slowperiod = 26  # MACD Slow EMA period
    signalperiod = 9  # Signal period
    stop_loss_pct = 0.02  # Stop loss 2%
    take_profit_pct = 0.04  # Take profit 4%

    def init(self):
        # Tính MACD và Signal
        self.macd, self.signal, _ = self.I(talib.MACD, self.data.Close, fastperiod=self.fastperiod, 
                                           slowperiod=self.slowperiod, signalperiod=self.signalperiod)

    def next(self):
        # Điều kiện mua: MACD cắt lên Signal và MACD > 0
        if crossover(self.macd, self.signal) and self.macd[-1] > 0:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)  # Stop loss là 2% dưới giá hiện tại
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)  # Take profit là 4% trên giá hiện tại
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán: MACD cắt xuống Signal và MACD < 0
        elif crossover(self.signal, self.macd) and self.macd[-1] < 0:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)  # Stop loss là 2% trên giá hiện tại
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)  # Take profit là 4% dưới giá hiện tại
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho Momentum Trading MACD
def run_backtest_optimize_momentum_macd(train_data, val_data, fastperiod_range, slowperiod_range, signalperiod_range):
    print(f"Optimizing MomentumTradingMACD on training set...")
    bt_train = Backtest(train_data, MomentumTradingMACD, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        fastperiod=fastperiod_range,  # Phạm vi tối ưu fastperiod
        slowperiod=slowperiod_range,  # Phạm vi tối ưu slowperiod
        signalperiod=signalperiod_range,  # Phạm vi tối ưu signalperiod
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MomentumTradingMACD, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
fastperiod_range = range(8, 13, 1)
slowperiod_range = range(20, 31, 2)
signalperiod_range = range(7, 12, 1)

stats_train_macd, stats_val_macd = run_backtest_optimize_momentum_macd(train_hour, valid_hour, fastperiod_range, slowperiod_range, signalperiod_range)


Optimizing MomentumTradingMACD on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [390]:
stats_train_macd

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   60.588586
Equity Final [$]                     1611.981
Equity Peak [$]                        3000.0
Return [%]                           -46.2673
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -14.396269
Volatility (Ann.) [%]                16.82589
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -59.185467
Avg. Drawdown [%]                  -59.185467
Max. Drawdown Duration     1459 days 01:00:00
Avg. Drawdown Duration     1459 days 01:00:00
# Trades                                  149
Win Rate [%]                        43.624161
Best Trade [%]                       4.531698
Worst Trade [%]                     -4.115325
Avg. Trade [%]                    

In [391]:
stats_val_macd

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   59.427208
Equity Final [$]                     2266.566
Equity Peak [$]                      3841.018
Return [%]                           -24.4478
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -13.079231
Volatility (Ann.) [%]               19.546939
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -50.368496
Avg. Drawdown [%]                   -5.566982
Max. Drawdown Duration      549 days 00:00:00
Avg. Drawdown Duration       42 days 09:00:00
# Trades                                   72
Win Rate [%]                        45.833333
Best Trade [%]                       3.775424
Worst Trade [%]                      -3.48201
Avg. Trade [%]                    

### Momentum trading with RSI and MA

In [398]:
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Chiến lược Momentum Trading với RSI và MA
class MomentumTradingRSIMA(Strategy):
    ma_short = 20  # MA ngắn hạn
    ma_long = 50  # MA dài hạn
    rsi_period = 14  # RSI period
    stop_loss_pct = 0.02  # Stop loss 2%
    take_profit_pct = 0.04  # Take profit 4%

    def init(self):
        # Tính các đường MA và RSI
        self.ma_short = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_short)
        self.ma_long = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_long)
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period)

    def next(self):
        # Điều kiện mua: RSI < 30 và giá dưới MA dài hạn
        if self.rsi[-1] < 30 and self.data.Close[-1] < self.ma_long[-1]:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán: RSI > 70 và giá trên MA dài hạn
        elif self.rsi[-1] > 70 and self.data.Close[-1] > self.ma_long[-1]:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho RSI + MA
def run_backtest_optimize_rsi_ma(train_data, val_data, ma_short_range, ma_long_range, rsi_period_range):
    print(f"Optimizing MomentumTradingRSIMA on training set...")
    bt_train = Backtest(train_data, MomentumTradingRSIMA, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        ma_short=ma_short_range,  # Phạm vi tối ưu MA ngắn hạn
        ma_long=ma_long_range,  # Phạm vi tối ưu MA dài hạn
        rsi_period=rsi_period_range,  # Phạm vi tối ưu RSI period
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MomentumTradingRSIMA, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
ma_short_range = range(10, 31, 5)
ma_long_range = range(40, 101, 10)
rsi_period_range = range(10, 21, 5)

stats_train_rsi_ma, stats_val_rsi_ma = run_backtest_optimize_rsi_ma(train_hour, valid_hour, ma_short_range, ma_long_range, rsi_period_range)


Optimizing MomentumTradingRSIMA on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

Backtesting on validation set...


In [399]:
stats_train_rsi_ma

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   40.962418
Equity Final [$]                      743.274
Equity Peak [$]                      3941.951
Return [%]                           -75.2242
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -29.472826
Volatility (Ann.) [%]               24.798238
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -81.144514
Avg. Drawdown [%]                  -11.968562
Max. Drawdown Duration     1414 days 02:00:00
Avg. Drawdown Duration      160 days 17:00:00
# Trades                                  231
Win Rate [%]                        41.558442
Best Trade [%]                       3.359915
Worst Trade [%]                     -6.425278
Avg. Trade [%]                    

In [400]:
stats_val_rsi_ma

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                   25.019889
Equity Final [$]                      851.038
Equity Peak [$]                        3000.0
Return [%]                         -71.632067
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -46.738444
Volatility (Ann.) [%]               14.323507
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -73.025267
Avg. Drawdown [%]                  -73.025267
Max. Drawdown Duration      716 days 05:00:00
Avg. Drawdown Duration      716 days 05:00:00
# Trades                                   86
Win Rate [%]                        18.604651
Best Trade [%]                       3.189383
Worst Trade [%]                     -5.276704
Avg. Trade [%]                    

### Momentum trading with CCI and MA

In [395]:
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Chiến lược Momentum Trading với CCI và MA
class MomentumTradingCCI(Strategy):
    cci_period = 20  # Tham số mặc định cho CCI
    ma_period = 50  # Tham số mặc định cho MA dài hạn
    stop_loss_pct = 0.02  # Stop loss 2%
    take_profit_pct = 0.04  # Take profit 4%

    def init(self):
        # Tính MA và CCI
        self.ma = self.I(talib.SMA, self.data.Close, timeperiod=self.ma_period)
        self.cci = self.I(talib.CCI, self.data.High, self.data.Low, self.data.Close, timeperiod=self.cci_period)

    def next(self):
        # Điều kiện mua: CCI < -100 và giá dưới MA
        if self.cci[-1] < -100 and self.data.Close[-1] < self.ma[-1]:
            sl = self.data.Close[-1] * (1 - self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 + self.take_profit_pct)
            self.buy(size=1/6, sl=sl, tp=tp)

        # Điều kiện bán: CCI > 100 và giá trên MA
        elif self.cci[-1] > 100 and self.data.Close[-1] > self.ma[-1]:
            sl = self.data.Close[-1] * (1 + self.stop_loss_pct)
            tp = self.data.Close[-1] * (1 - self.take_profit_pct)
            self.sell(size=1/6, sl=sl, tp=tp)

# Hàm backtest và tối ưu hóa cho CCI + MA
def run_backtest_optimize_cci_ma(train_data, val_data, cci_period_range, ma_period_range):
    print(f"Optimizing MomentumTradingCCI on training set...")
    bt_train = Backtest(train_data, MomentumTradingCCI, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    
    stats_train = bt_train.optimize(
        cci_period=cci_period_range,  # Phạm vi tối ưu CCI period
        ma_period=ma_period_range,  # Phạm vi tối ưu MA period
        maximize='Return [%]',  # Tối ưu dựa trên lợi nhuận
        random_state=42,
        method='grid',
        return_heatmap=False
    )

    # Backtest trên tập validation
    print("Backtesting on validation set...")
    bt_val = Backtest(val_data, MomentumTradingCCI, cash=3_000, commission=0.01, margin=0.13, hedging=True)
    stats_val = bt_val.run()

    return stats_train, stats_val

# Example usage:
cci_period_range = range(14, 31, 5)
ma_period_range = range(30, 101, 10)

stats_train_cci_ma, stats_val_cci_ma = run_backtest_optimize_cci_ma(train_hour, valid_hour, cci_period_range, ma_period_range)


Optimizing MomentumTradingCCI on training set...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting on validation set...


In [396]:
stats_train_cci_ma

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                   37.204216
Equity Final [$]                      510.273
Equity Peak [$]                       3487.35
Return [%]                           -82.9909
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -35.808231
Volatility (Ann.) [%]                19.47293
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -85.367887
Avg. Drawdown [%]                  -20.456549
Max. Drawdown Duration     1414 days 02:00:00
Avg. Drawdown Duration      289 days 11:00:00
# Trades                                  227
Win Rate [%]                        37.004405
Best Trade [%]                       3.392516
Worst Trade [%]                     -6.208026
Avg. Trade [%]                    

In [397]:
stats_val_cci_ma

Start                     2022-08-25 13:00:00
End                       2024-08-30 14:00:00
Duration                    736 days 01:00:00
Exposure Time [%]                    8.750994
Equity Final [$]                      742.598
Equity Peak [$]                        3000.0
Return [%]                         -75.246733
Buy & Hold Return [%]                2.760429
Return (Ann.) [%]                  -50.247345
Volatility (Ann.) [%]               14.638895
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -75.773733
Avg. Drawdown [%]                  -75.773733
Max. Drawdown Duration      716 days 05:00:00
Avg. Drawdown Duration      716 days 05:00:00
# Trades                                   87
Win Rate [%]                        16.091954
Best Trade [%]                        3.29461
Worst Trade [%]                     -5.276704
Avg. Trade [%]                    